In [1]:
import os
import argparse
import datetime

import torch
from torch import nn
import numpy as np
import gymnasium as gym

from tianshou.data import Collector, Batch, to_torch
from tianshou.data.types import RolloutBatchProtocol
from tianshou.data.buffer.vecbuf import VectorReplayBuffer, ReplayBuffer
from tianshou.env import SubprocVectorEnv
from tianshou.policy import SACPolicy, BasePolicy
from tianshou.utils.net.common import Net
from tianshou.utils.net.continuous import ActorProb, Critic
from examples.offline.utils import load_buffer_d4rl
from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import TensorboardLogger
from tianshou.trainer import OffpolicyTrainer

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:596: UserWarning: WARN: plugin: shimmy.registration:register_gymnasium_envs raised Traceback (most recent call last):
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py", line 594, in load_plugin_envs
    fn()
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/shimmy/registration.py", line 262, in register_gymnasium_envs
    _register_dm_control_envs()
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/shimmy/registration.py", line 26, in _register_dm_control_envs
    from shimmy.dm_control_compatibility import DmControlCompatibilityV0
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/shim

In [2]:
device = "cuda:3"

In [3]:
gym.envs.registry

{'CartPole-v0': EnvSpec(id='CartPole-v0', entry_point='gymnasium.envs.classic_control.cartpole:CartPoleEnv', reward_threshold=195.0, nondeterministic=False, max_episode_steps=200, order_enforce=True, autoreset=False, disable_env_checker=False, apply_api_compatibility=False, kwargs={}, namespace=None, name='CartPole', version=0, additional_wrappers=(), vector_entry_point='gymnasium.envs.classic_control.cartpole:CartPoleVectorEnv'),
 'CartPole-v1': EnvSpec(id='CartPole-v1', entry_point='gymnasium.envs.classic_control.cartpole:CartPoleEnv', reward_threshold=475.0, nondeterministic=False, max_episode_steps=500, order_enforce=True, autoreset=False, disable_env_checker=False, apply_api_compatibility=False, kwargs={}, namespace=None, name='CartPole', version=1, additional_wrappers=(), vector_entry_point='gymnasium.envs.classic_control.cartpole:CartPoleVectorEnv'),
 'MountainCar-v0': EnvSpec(id='MountainCar-v0', entry_point='gymnasium.envs.classic_control.mountain_car:MountainCarEnv', reward_t

In [3]:
def get_sac_args():
    args = argparse.Namespace(
        task="Hopper-v2",
        buffer_size=1000000,
        hidden_sizes=[256, 256, 256],
        actor_lr=1e-4,
        critic_lr=3e-4,
        gamma=0.99,
        tau=0.005,
        alpha_lr=1e-4,
        alpha=0.36,
        start_timesteps=1,
        epoch=200,
        step_per_epoch=5000,
        step_per_collect=1,
        update_per_step=1,
        batch_size=256,
        training_num=1,
        test_num=10,
        device=device,
        norm_layer=True,
    )
    return args

In [4]:
args = get_sac_args()
env = gym.make(args.task)

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment Hopper-v2 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/mujoco/mujoco_env.py:211: DeprecationWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.deprecation(


In [5]:
def load_policy(path):
    args = get_sac_args()
    env = gym.make(args.task)
    args.state_shape = env.observation_space.shape or env.observation_space.n
    args.action_shape = env.action_space.shape or env.action_space.n
    args.max_action = env.action_space.high[0]
    # model
    net_a = Net(args.state_shape, hidden_sizes=args.hidden_sizes, device=args.device)
    actor = ActorProb(
        net_a,
        args.action_shape,
        device=args.device,
        unbounded=True,
        conditioned_sigma=True,
    ).to(args.device)
    actor_optim = torch.optim.Adam(actor.parameters(), lr=args.actor_lr)
    net_c1 = Net(
        args.state_shape,
        args.action_shape,
        hidden_sizes=args.hidden_sizes,
        concat=True,
        device=args.device,
        norm_layer=nn.LayerNorm if args.norm_layer else None
    )
    net_c2 = Net(
        args.state_shape,
        args.action_shape,
        hidden_sizes=args.hidden_sizes,
        concat=True,
        device=args.device,
        norm_layer=nn.LayerNorm if args.norm_layer else None
    )
    critic1 = Critic(net_c1, device=args.device).to(args.device)
    critic1_optim = torch.optim.Adam(critic1.parameters(), lr=args.critic_lr)
    critic2 = Critic(net_c2, device=args.device).to(args.device)
    critic2_optim = torch.optim.Adam(critic2.parameters(), lr=args.critic_lr)
    target_entropy = -np.prod(env.action_space.shape)
    log_alpha = torch.tensor([np.log(args.alpha)], requires_grad=True, device=device)
    alpha_optim = torch.optim.Adam([log_alpha], lr=args.alpha_lr)
    alpha = (target_entropy, log_alpha, alpha_optim)
    policy = SACPolicy(
        actor,
        actor_optim,
        critic1,
        critic1_optim,
        critic2,
        critic2_optim,
        tau=args.tau,
        gamma=args.gamma,
        alpha=alpha,
        action_space=env.action_space,
    )
    policy.load_state_dict(torch.load(path, map_location=args.device))
    print("Loaded agent from: ", path)
    return policy

In [58]:
cql_policy = load_policy("/data/user/R901105/dev/log/Hopper-v2/cql/0/231220-111219/policy.pth")

Loaded agent from:  /data/user/R901105/dev/log/Hopper-v2/cql/0/231220-111219/policy.pth


In [6]:
sac_policy = load_policy("/data/user/R901105/dev/log/Hopper-v2/sac/0/231219-163624/policy.pth")

Loaded agent from:  /data/user/R901105/dev/log/Hopper-v2/sac/0/231219-163624/policy.pth


In [82]:
from tianshou.env.gym_wrappers import ContinuousToDiscrete
num_actions = 10

env = SubprocVectorEnv([lambda: ContinuousToDiscrete(gym.make(args.task), num_actions) for _ in range(20)])

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment Hopper-v2 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/mujoco/mujoco_env.py:211: DeprecationWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.deprecation(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment Hopper-v2 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/e

In [83]:
def reverse_action(act: np.ndarray) -> np.ndarray:
    _env = gym.make(args.task)
    low, high = _env.action_space.low, _env.action_space.high
    action_per_dim = [num_actions] * _env.action_space.shape[0]
    mesh = np.array(
        [np.linspace(lo, hi, a) for lo, hi, a in zip(low, high, action_per_dim)],
        dtype=object
    )
    # If act is a PyTorch tensor, convert it to a numpy array
    if torch.is_tensor(act):
        act = act.cpu().numpy()
    # modify act
    assert len(act.shape) <= 2, f"Unknown action format with shape {act.shape}."
    if len(act.shape) == 1:
        return np.array([np.argmin(np.abs(mesh[i] - a)) for i, a in enumerate(act)])
    return np.array([[np.argmin(np.abs(mesh[i] - a)) for i, a in enumerate(a_)] for a_ in act])

In [84]:
class SPIPolicy(BasePolicy):
    def __init__(self, sac_policy, num_actions, random_p=0.25):
        super().__init__() 
        self.sac_policy = sac_policy
        self.num_actions = num_actions
        self.random_p = random_p

    def forward(self, batch, state=None, **kwargs):
        sac_act = self.sac_policy(batch, state, **kwargs).act
        sac_act = reverse_action(sac_act)
        # random action
        act = np.zeros_like(sac_act)
        for i in range(len(act)):
            act[i] = np.random.choice(self.num_actions)
        for i in range(len(act)):
            if np.random.rand() > self.random_p:
                act[i] = sac_act[i]
        return Batch(act=act)
    
    def learn(self, batch, **kwargs):
        return {}
    
    def train(self, mode: bool = True) -> "SPIPolicy":
        self.sac_policy.train(mode)
        return self

In [90]:
spi_policy = SPIPolicy(sac_policy, num_actions, random_p=0.05)

In [93]:
spi_buffer = VectorReplayBuffer(100000, 20)

In [94]:
spi_policy.eval()
spi_collector = Collector(spi_policy, env, spi_buffer)
spi_result = spi_collector.collect(n_episode=1000)
spi_result

{'n/ep': 1000,
 'n/st': 454727,
 'rews': array([ 466.29402436,  534.00083536,  787.97912797,  751.56875933,
         829.85546261,  835.00265311,  967.63967629,  992.2478825 ,
        1007.60279812, 1300.96996581, 1334.51360064, 1320.56080531,
        1598.8445459 , 1636.69301338,  764.02490777, 1827.89017545,
        2018.69810302, 2013.78579056, 1097.94501355,  766.42761231,
        1017.996668  , 1398.07446859,  975.86828863, 1778.72981464,
        1978.67753744,  686.91904831, 1548.51003024,  746.29481742,
        1347.98738655, 3145.07010841, 3175.88056512, 2536.24148643,
        1730.43598641, 2165.30080499, 1956.68293232, 3524.01153751,
         730.58407188, 2644.70760943, 1736.08733795, 1977.63401709,
        1023.80944575, 1936.1691252 ,  843.53111992, 1676.21517354,
         690.46404973, 3549.79481835,  940.82862075, 1071.6732077 ,
         739.77473389,  834.6719128 , 2734.53733099, 3011.41920937,
        1585.02261023, 2827.40720643, 2388.54765838, 1934.41877052,
        

In [96]:
spi_buffer.save_hdf5("spi_buffer.hdf5")

In [49]:
envs = SubprocVectorEnv([lambda: gym.make(args.task) for _ in range(10)])

In [50]:
cql_policy.eval()
cql_collector = Collector(cql_policy, envs)
cql_result = cql_collector.collect(n_episode=40)
cql_result

{'n/ep': 40,
 'n/st': 28438,
 'rews': array([1875.527446  , 1985.02329145, 2076.8943044 , 2169.54324235,
        2220.29535078, 2354.63864142, 2632.09455398, 2773.12230279,
        2846.51336189, 3324.07538659, 1513.33676209, 1412.20228822,
        2114.93811745, 2183.43096061, 2097.08056415, 2691.77921814,
        1813.60546139, 2448.4467662 , 2040.81795527, 2051.75918271,
        3283.1869963 , 3282.86971813, 1459.23786367, 1888.89257348,
        2685.39992789, 1867.88289683, 2976.89779533, 3031.58666299,
        2150.55391171, 2744.34696107, 2319.66226352, 3271.27851469,
        1871.13751454, 2172.16562337, 2238.02661416, 2573.74728189,
        1605.1772629 , 2721.9337948 , 3075.98032085, 2501.72036928]),
 'lens': array([ 566,  605,  630,  653,  669,  709,  790,  832,  851, 1000,  460,
         439,  627,  648,  638,  809,  552,  738,  613,  618, 1000, 1000,
         446,  574,  807,  563,  890,  904,  645,  818,  690, 1000,  563,
         656,  673,  776,  491,  818,  919,  758]),

In [51]:
sac_policy.eval()
sac_collector = Collector(sac_policy, envs)
sac_result = sac_collector.collect(n_episode=40)
sac_result

{'n/ep': 40,
 'n/st': 39312,
 'rews': array([3490.20986974, 3684.70606198, 3591.47819272, 3597.37233087,
        3538.29855872, 3591.69464872, 3520.78922286, 3586.09690681,
        3588.96709079, 3575.0735309 , 2592.20752796, 3528.43318319,
        3657.0264336 , 3555.0225114 , 3555.33762397, 3567.77300397,
        3541.87593344, 3712.8363668 , 3585.36534082, 3566.03740618,
        3596.09974697, 3376.00707725, 3588.74366652, 3560.44204471,
        3554.94183537, 3548.74198787, 3578.99220746, 3556.44991665,
        3535.1903398 , 3556.28216648, 3566.60430196, 2529.72685258,
        3524.95554888, 3556.10858374, 3562.88478265, 3559.90796793,
        3591.3873296 , 3551.17067614, 3513.85777727, 3563.2662359 ]),
 'lens': array([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,  701,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,  898,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,  713, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000]),

In [81]:
from tianshou.data import ReplayBuffer
from tianshou.data.types import RolloutBatchProtocol


class MixedPolicy(BasePolicy):
    def __init__(self, base_policy: BasePolicy, expert_policy: BasePolicy, action_space, buffer):
        super().__init__(action_space=action_space, action_scaling=True)
        self.base_policy = base_policy
        self.expert_policy = expert_policy
        self.device = device
        self.buffer = buffer

    def forward(self, batch: RolloutBatchProtocol, state=None, **kwargs):
        bsz = len(batch.obs)
        batch = to_torch(batch, dtype=torch.float32, device=self.device)
        self.base_policy.eval()
        self.expert_policy.eval()
        with torch.no_grad():
            expert_result = self.expert_policy(batch)
            base_result = self.base_policy(batch)
            expert_qvalues1 = self.expert_policy.critic1(batch.obs, expert_result.act)
            expert_qvalues2 = self.expert_policy.critic2(batch.obs, expert_result.act)
            expert_qvalues = torch.minimum(expert_qvalues1, expert_qvalues2)
            base_qvalues1 = self.base_policy.critic1(batch.obs, base_result.act)
            base_qvalues2 = self.base_policy.critic2(batch.obs, base_result.act)
            base_qvalues = torch.minimum(base_qvalues1, base_qvalues2)
        cede_ctrl = base_qvalues < expert_qvalues
        actions = torch.where(cede_ctrl, expert_result.act, base_result.act)
        return Batch(**{'act': actions, 'policy': Batch({'cede_ctrl': cede_ctrl})})

    def train(self, mode: bool = True) -> "MixedPolicy":
        self.base_policy.train(mode)
        return self
    
    def process_fn(self, batch: RolloutBatchProtocol, buffer: ReplayBuffer, indices: np.ndarray) -> RolloutBatchProtocol:
        return self.base_policy.process_fn(batch, buffer, indices)

    def learn(self, batch, **kwargs):
        # cede_ctrl = batch.policy.cede_ctrl.cpu().squeeze()
        # train_batch = batch[~cede_ctrl]
        # info = self.base_policy.learn(train_batch)
        info = self.base_policy.learn(batch)
        return info

In [11]:
test_buffer = VectorReplayBuffer(5000, 5)

In [82]:
mixed_policy = MixedPolicy(cql_policy, sac_policy, env.action_space, test_buffer)

In [12]:
mixed_collector = Collector(mixed_policy, envs)
mixed_result = mixed_collector.collect(n_episode=40)
mixed_result

{'n/ep': 40,
 'n/st': 39442,
 'rews': array([2877.85420345, 3579.38976417, 3566.2887685 , 3612.26257751,
        3648.73333248, 3655.45149677, 3559.89113674, 3552.6419363 ,
        3588.0665408 , 3578.10143437, 3576.98877615, 3561.46943533,
        3558.70658515, 3598.58161738, 3558.16991895, 3525.51417879,
        3579.98865515, 3558.74737203, 3518.58784384, 3641.33416581,
        3563.76800552, 3231.92112228, 3558.71492095, 3535.73037386,
        3558.96929922, 3583.61728139, 3556.20513785, 3542.30987626,
        3584.5683532 , 3573.77252436, 2903.35971768, 3574.50541953,
        3551.28549583, 3540.74278544, 3513.75433021, 3546.94601085,
        3569.82593565, 3520.37927402, 3546.47417846, 3562.03266267]),
 'lens': array([ 798, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,  869,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,  775, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000]),

In [59]:
class DeterministicPolicy(BasePolicy):
    def __init__(self, policy, action_space):
        super().__init__(action_space=action_space, action_scaling=True)
        self.policy = policy

    def train(self, mode: bool = True) -> "DeterministicPolicy":
        self.policy.train(mode)
        return self
    
    def forward(self, batch, state=None, **kwargs):
        self.policy.eval()
        return self.policy(batch)
    
    def learn(self, batch, **kwargs):
        info = self.policy.learn(batch)
        return info
    
    def process_fn(self, batch: RolloutBatchProtocol, buffer: ReplayBuffer, indices: np.ndarray) -> RolloutBatchProtocol:
        return self.policy.process_fn(batch, buffer, indices)

In [60]:
det_cql = DeterministicPolicy(cql_policy, env.action_space)

In [ ]:
det_cql_collector = Collector(cql_policy, envs)
cql_result = cql_collector.collect(n_episode=40)
cql_result

In [52]:
test_envs = SubprocVectorEnv([lambda: gym.make(args.task) for _ in range(5)])

In [13]:
offline_data = load_buffer_d4rl("hopper-medium-v2")

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gym/envs/mujoco/mujoco_env.py:190: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
load datafile: 100%|██████████| 21/21 [00:01<00:00, 16.48it/s]


In [53]:
policy = cql_policy

In [54]:
test_buffer = VectorReplayBuffer(5000, 5)
train_collector = Collector(policy, env, offline_data)
test_collector = Collector(policy, test_envs, test_buffer)

/data/user/R901105/dev/my_fork/tianshou/tianshou/data/collector.py:70: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


In [55]:
# log
now = datetime.datetime.now().strftime("%y%m%d-%H%M%S")
log_name = os.path.join(args.task, policy.__class__.__name__, now)
log_path = os.path.join("/data/user/R901105/dev/log", log_name)
writer = SummaryWriter(log_path)
logger = TensorboardLogger(writer)
print(log_path)

/data/user/R901105/dev/log/Hopper-v2/SACPolicy/240410-124843


In [15]:
def get_returns(policy, return_cede_ctrl=False):
    policy.eval()
    returns = []
    cede_ctrl = []
    for _ in range(5):
        obs, info = env.reset()
        done = False
        cum_reward = 0
        actions = []
        if return_cede_ctrl:
            cctrl_ep = []
        while not done:
            batch = Batch(obs=torch.from_numpy(np.expand_dims(obs, 0)).to(device), info=info)
            with torch.no_grad():
                result = policy(batch)
            act = result.act.cpu().squeeze().numpy()
            act = policy.map_action(act)
            if return_cede_ctrl:
                cctrl_ep.append(result.policy.cede_ctrl.cpu().squeeze().numpy())
            obs, reward, terminated, truncated, info = env.step(act)
            actions.append(act)
            cum_reward += reward
            done = terminated or truncated
        returns.append(cum_reward)
        if return_cede_ctrl:
            cede_ctrl.append(np.array(cctrl_ep).mean())
    if return_cede_ctrl:
        return np.array(returns),  np.array(cede_ctrl).mean()
    return np.array(returns), _

In [16]:
def test_fn(num_epoch: int, step_idx: int):
    returns, cede_ctrl = get_returns(policy, True)
    print(returns.mean(), cede_ctrl.mean())

In [56]:
result = OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    # test_fn=test_fn,
    max_epoch=200,
    step_per_epoch=1000,
    step_per_collect=1,
    episode_per_test=1,
    batch_size=256,
    logger=logger,
    update_per_step=1,
    test_in_train=False,
).run()

Epoch #1: 1001it [00:15, 65.10it/s, alpha=0.322, env_step=1000, gradient_step=1000, len=886, loss/actor=-265.550, loss/alpha=-1.427, loss/critic1=10.221, loss/critic2=10.537, n/ep=0, n/st=1, rew=2811.60]                          


Epoch #1: test_reward: 3260.866312 ± 0.000000, best_reward: 3260.866312 ± 0.000000 in #1


Epoch #2: 1001it [00:15, 65.87it/s, alpha=0.289, env_step=2000, gradient_step=2000, len=611, loss/actor=-262.574, loss/alpha=-1.800, loss/critic1=5.124, loss/critic2=5.014, n/ep=0, n/st=1, rew=2081.29]                           


Epoch #2: test_reward: 1326.888789 ± 0.000000, best_reward: 3260.866312 ± 0.000000 in #1


Epoch #3: 1001it [00:15, 65.20it/s, alpha=0.260, env_step=3000, gradient_step=3000, len=480, loss/actor=-260.373, loss/alpha=-2.026, loss/critic1=3.925, loss/critic2=3.763, n/ep=0, n/st=1, rew=1496.70]                            


...

Epoch #47: test_reward: 1156.083544 ± 0.000000, best_reward: 3329.365953 ± 0.000000 in #14


Epoch #48: 1001it [00:15, 64.77it/s, alpha=0.033, env_step=48000, gradient_step=48000, len=170, loss/actor=-283.064, loss/alpha=0.107, loss/critic1=12.433, loss/critic2=12.526, n/ep=0, n/st=1, rew=419.88]                           


Epoch #48: test_reward: 424.777489 ± 0.000000, best_reward: 3329.365953 ± 0.000000 in #14


Epoch #49:  16%|#6        | 160/1000 [00:02<00:12, 64.64it/s, alpha=0.033, env_step=48160, gradient_step=48160, len=170, loss/actor=-284.213, loss/alpha=0.576, loss/critic1=21.312, loss/critic2=21.094, n/ep=0, n/st=1, rew=419.88]


KeyboardInterrupt: 